## Oppgave 2

In [18]:
import ipywidgets as widgets
from ipywidgets import interactive

from scipy import ndimage
from skimage import io,util,color

from skimage import feature
from skimage.filters import sobel

import numpy


In [2]:
def readAndShowImage(image):
    im = io.imread(image)
    io.imshow(im)
    io.show()

inputImage = widgets.Dropdown(
    options={
        'Strand':'strand.jpeg',
        'Mandrill':'mandrill.png',
        'Trump':'trump.jpg',
        'Gruppe':'gruppe.jpg',
        'RGB':"rgb.png",
        'Dress':'TheDress.png'
        },
    description='Bilde: ',
    disabled=False,
)

interactive(readAndShowImage, image=inputImage)

interactive(children=(Dropdown(description='Bilde: ', options={'Strand': 'strand.jpeg', 'Mandrill': 'mandrill.…

Kode for marr-hildreth kantdeteksjon

In [15]:
def smooth(image, sigma):
    # Read in image and convert to appropriate format
    #image = util.img_as_float(color.rgb2gray(io.imread(image)))

    # Read in sigma 
    sigma = float(sigma)

    # Calculate size of filter mask
    size = int(6*sigma)-1

    # A function for converting top-left aligned coordinates to center aligned 
    # coordinates
    def dist(i,j):
      # Find the center index
      center = size//2
      # Return the converted coordinates
      return (i-center,j-center) 

    # A function for creating a Gaussian smoothing filter
    def gaussian():
      # Create the filter image
      mask = numpy.ndarray((size,size), dtype=numpy.float) 

      # Calculate the constant factor of the Gaussian equation. Not needed if we
      # are normalizing the filter after calculations
      constant = 1/(2*numpy.pi*sigma**2)

      # Loop through all the pixels of the mask
      for i in range(0,size):
        for j in range(0,size):
          # Find the center-aligned coordinates for the current pixel
          x,y = dist(i,j)
          # Calculate the pixel value 
          mask[i,j] = constant * numpy.exp(-(x**2 + y**2)/(2*sigma**2))


      return mask

    # Create the Gaussian filter
    gaussian_filter = gaussian()

    # Smooth the image using built-in convolution operator and our filter
    smooth = ndimage.convolve(image, gaussian_filter, mode="nearest")

    return smooth

def laplacian(image):
    laplacian_filter = numpy.asarray([[-1,-1,-1],[-1,8,-1],[-1,-1,-1]])
    return ndimage.convolve(image, laplacian_filter, mode="nearest")

def zeroCrossPair(pixel1,pixel2):
    if(pixel1*pixel2 < 0):
        return abs(pixel1 - pixel2)
    else:
        return 0

def zeroCrossing3x3(subim, tresh):
    ret = 0
    for x in range(3):
        ret += zeroCrossPair(subim[0,x],subim[2,2-x]) 
    ret += zeroCrossPair(subim[1,0],subim[1,2]) 
    return ret

def zeroCrossing(image,tresh):
    height,width = image.shape
    ret = numpy.zeros(image.shape, dtype=bool)
    for i in range(1,height-1):
        for j in range(1,width-1):
            subim = image[i-1:i+2,j-1:j+2]
            ret[i,j] = zeroCrossing3x3(subim,tresh)
    return ret > tresh

def marr_hildreth(im,sigma,tresh):
    return zeroCrossing(laplacian(smooth(im,sigma)),tresh)

Dette er koden for å sammenligne canny kantdeteksjon med sobel og marr hildreth.

In [38]:
def update(image,tresh,sigma,lowHigh):
    im = io.imread(image, as_gray=True)

    edge_canny = feature.canny(im,sigma=sigma,low_threshold=lowHigh[0], high_threshold=lowHigh[1])
    io.imshow(edge_canny)
    io.show()

    edge_sobel = sobel(im)
    io.imshow(edge_sobel > tresh, cmap='gray')
    io.show()

    edge_marr = marr_hildreth(im,sigma,tresh)
    io.imshow(edge_marr, cmap='gray')
    io.show()
   
highLowSlider = widgets.FloatRangeSlider(
    value=[0.1, 0.3],
    min=0,
    max=0.7,
    step=0.01,
    description='Low High Hysterisis:',
    continuous_update=False,
    orientation='horizontal'
)

interactive(update, image=inputImage, tresh=0.05, sigma=1.5, lowHigh=highLowSlider)

interactive(children=(Dropdown(description='Bilde: ', index=3, options={'Strand': 'strand.jpeg', 'Mandrill': '…

Som man kan se får man med sobel filteret med mesteparten av kantene i bildet, men kantene blir ganske tykke. med marr-hildreth får man for mange kanter, men kantene har en fast tykkelse. med canny kantdeteksjon vil jeg si at noen av kantene forsvinner, men dette er mulig å fikse med bedre parametre, kantene er også 1 pixel tykke noe som gjør de enkle å jobbe med.